In [44]:
import sympy as sym
import math as m
import numpy as np
import pandas as pd

import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import SIC_POVM_functions as sic

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


In [45]:
#Class definitions
class seed_sol:
    def __init__(self, seed):
        self.seed = seed
    @property
    def initial_guess(self):
        np.random.seed(self.seed)
        initial_guess = np.random.rand(35)  # random initial guess
        return initial_guess
    @property
    def solution(self):
        solution = fsolve(Lagrange_eqs, self.initial_guess)
        return solution
    @property
    def res_list(self):
        res_list = Lagrange_eqs(self.solution)
        return res_list
    @property
    def res(self):
        res = np.sum(np.abs(self.res_list))
        return res
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        sol = self.solution
        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7
        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy 
        for i in range(2):                              # first two vecs are known
            POVM_normalized_9d[i].extend([0] *4)
        for part in solution_add:                       # adding (already normalized) elements from the solution to make the remaining vectors 9d
            for i in range(7):
                # POVM_normalized_9d[i+2].append(part[i])
                POVM_normalized_9d[i+2].append((1/np.sqrt(6))*(part[i]))        # normalizing coming from the fixing of the lagrangian eqns
        return POVM_normalized_9d


class POVM_relations:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
        self.inner_products = {}
    def dot_product(self, i, j):
        if i < 0 or j < 0 or i > 8 or j > 8:
            raise ValueError('Bad index, must be integer 0 to 8')
        return np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])
    def all(self, clean = True, threshold = 1e-12, absolutes = True):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        for i in range(9):
            for j in range(i, 9):
                key = f'{i+1}{j+1}'
                value = self.dot_product(i, j)
                self.inner_products[key] = value
        #cleaning the dictionary
        rel = self.inner_products
        for key in rel:
            if absolutes == True:
                rel[key] = abs(rel[key])
            if abs(rel[key]) < threshold:
                rel[key] = 0
        return rel


### Fixing the normalization problem from Lagrange Equations, and associated stuff

In [46]:
# creating five lists POVMs
w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = [[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]             # unnormalized POVM direction vectors
# POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors
POVM_vec = (1/np.sqrt(2))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors

# c4j_list = [2,.5,.5,.5, (-.25-.433013j), -.25, .5, -.25, (-.25-.433013j)]           # fourth elements, not normalized yet. See sic.py for calcs
c4j_list = [2,.5,.5,.5, (-.25-.25j*np.sqrt(3)), (-.25+.25j*np.sqrt(3)), .5, (-.25+.25j*np.sqrt(3)), (-.25-.25j*np.sqrt(3))]           # fourth elements, not normalized yet. See sic.py for calcs
# c5j_list = [0, np.sqrt(15)/2 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]
c5j_list = [0, np.sqrt(15)/2 , (1/np.sqrt(15))*(3/2), (1/np.sqrt(15))*(2*w**2-.5) ,-(1/np.sqrt(15))*(2+(5/2)*w**2), (1/np.sqrt(15))*(2-w/2), (1/np.sqrt(15))*(2*w-.5), -(1/np.sqrt(15))*(2+(5/2)*w), (1/np.sqrt(15))*(2-.5*w**2)]

POVM_unnormalized_5d = []
for i in range(len(POVM_unnormalized)):
	# Directly append c4j_list[i] and c5j_list[i] to the copies of lists in POVM_unnormalized
	vec_5d_i = POVM_unnormalized[i] + [c4j_list[i], c5j_list[i]]
	POVM_unnormalized_5d.append(vec_5d_i)

POVM_unnormalized_5d[0]    ## unnormalized 5-lists POVM vectors

POVM_normalized_5d = ((1/(6**.5))*(np.array(POVM_unnormalized_5d))).tolist()       # normalized 5-lists POVM vectors

#print and compare
# print(POVM_unnormalized_5d[0])	## UNnormalized 5-lists POVM vectors
# print(POVM_normalized_5d[0])	## normalized 5-lists POVM vectors

# delta_five  = [0.400000000000000, 0.433333333333333, 0.381250043028167, 0.400000000000000, 0.400000000000000, 0.383333333333333, 0.431250043028167]
# delta_five_new = [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.3812500430281667+0j), (0.40000000000000013-3.0923901048776238e-18j), (0.4+1.898287466587076e-18j), (0.38333333333333347-7.881938927040631e-19j), (0.4312500430281667+3.469446951953614e-18j)]
# print('delta_five_new = ', [np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(9)])
delta_five_fixed = [ np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(2,9)]
print('delta_five_fixed = ', delta_five_fixed)


delta_five_fixed =  [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.43333333333333335+1.0728477129103336e-18j), (0.43333333333333346-4.471148278649265e-19j), (0.4333333333333333+2.6275172237666522e-18j), (0.4333333333333335+2.267799535210488e-19j), (0.43333333333333335+6.251554030278836e-19j)]


#### The 1/6 in ortho eq after only the lambda term but not in others is one way of doing it, and prolly not the correct one. Trying this approach for now , later will try the one where 1/6 is multiplied to the coefficients too.

In [47]:
# the main objective function containing the Lagrange equations
def Lagrange_eqs(vars):
    # Unpacking variables, C63 to C99 and y3 to y9
    C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99,  y3, y4, y5, y6, y7, y8, y9 = vars
    # global delta_five
    # delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five
    global delta_five_new
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five_fixed

    # Full set of 28 equations for Cij terms, based on previous interactions
    equations = [
        # Derivatives with respect to C63 to C69
        (1/6)*(np.conj(C63)*y3*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64)*y4*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65)*y5*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9*(1/6)),
        # Continue with C73 to C79
        # Derivatives with respect to C73 to C79
        (1/6)*(np.conj(C73)*y3*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74)*y4*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75)*y5*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9*(1/6)),
        # Derivatives with respect to C83 to C89
        (1/6)*(np.conj(C83)*y3*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84)*y4*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85)*y5*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9*(1/6)),
        # Derivatives with respect to C93 to C99
        (1/6)*(np.conj(C93)*y3*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94)*y4*(1/6)), 
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95)*y5*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9*(1/6)),
        #Now the normalization equations
        (1/6)*(abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2) + delta_33 - 1,  
        (1/6)*(abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2) + delta_44 - 1,
        (1/6)*(abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2) + delta_55 - 1,  
        (1/6)*(abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2) + delta_66 - 1,
        (1/6)*(abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2) + delta_77 - 1,  
        (1/6)*(abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2) + delta_88 - 1,
        (1/6)*(abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2) + delta_99 - 1
    ]
    assert len(equations) == 35
    return equations

"""# Example Usage:



# ig below for np.seed = 2918 
# initial_guess =[0.98030858, 0.35567294, 0.99141023, 0.47111957, 0.34998446, 0.43261049, 0.30442865, 0.21036997, 0.39912775, 0.24321787, 0.91072068, 0.59991195, 0.39894914, 0.9238744 , 0.27633149, 0.9263308 , 0.02004467, 0.09815871, 0.03103966, 0.9009708 , 0.60346855, 0.30131413, 0.22667874, 0.99340599, 0.71874642, 0.71156703, 0.8234102 , 0.80459364, 0.77652217, 0.75734249, 0.06087287, 0.93906535, 0.92258927, 0.22224891, 0.0029041 ] 
initial_guess =[0.13645285, 0.84315108 ,0.12490685 ,0.10271676 ,0.27940578 ,0.98665734, 0.52459312, 0.41561316, 0.70258652, 0.06884098, 0.77664312, 0.84248407, 0.67034251, 0.45412921, 0.69501392, 0.69186904, 0.52676626, 0.45965484, 0.31382016, 0.70279726, 0.29404767, 0.95437514, 0.84051819, 0.87257289, 0.2450531,  0.71094717, 0.34003065, 0.7689673,  0.49019627, 0.6539388, 0.00374957, 0.24125156, 0.29499423, 0.17469819, 0.0711829 ]
print("Initial guess:", initial_guess)
# initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5   # Initial guess for the variables (Cij real and imaginary parts, y)
solution = fsolve(Lagrange_eqs, initial_guess)
residuals = Lagrange_eqs(solution)                   # residuals and residues sum
residuals_sum = np.sum(np.abs(residuals))

print("Solution to the system:", solution)
print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)"""

'# Example Usage:\n\n\n\n# ig below for np.seed = 2918 \n# initial_guess =[0.98030858, 0.35567294, 0.99141023, 0.47111957, 0.34998446, 0.43261049, 0.30442865, 0.21036997, 0.39912775, 0.24321787, 0.91072068, 0.59991195, 0.39894914, 0.9238744 , 0.27633149, 0.9263308 , 0.02004467, 0.09815871, 0.03103966, 0.9009708 , 0.60346855, 0.30131413, 0.22667874, 0.99340599, 0.71874642, 0.71156703, 0.8234102 , 0.80459364, 0.77652217, 0.75734249, 0.06087287, 0.93906535, 0.92258927, 0.22224891, 0.0029041 ] \ninitial_guess =[0.13645285, 0.84315108 ,0.12490685 ,0.10271676 ,0.27940578 ,0.98665734, 0.52459312, 0.41561316, 0.70258652, 0.06884098, 0.77664312, 0.84248407, 0.67034251, 0.45412921, 0.69501392, 0.69186904, 0.52676626, 0.45965484, 0.31382016, 0.70279726, 0.29404767, 0.95437514, 0.84051819, 0.87257289, 0.2450531,  0.71094717, 0.34003065, 0.7689673,  0.49019627, 0.6539388, 0.00374957, 0.24125156, 0.29499423, 0.17469819, 0.0711829 ]\nprint("Initial guess:", initial_guess)\n# initial_guess = [0.2] * 1

In [ ]:
# for i in range(2800,15000):
for i in range(1000):  #[2916,2917,2918, 4531, 4532, 4533, 6032,6033]+ list(range(1, 100)):
    np.random.seed(i)
    initial_guess = np.random.rand(35)
    solution = fsolve(Lagrange_eqs, initial_guess)
    residuals = Lagrange_eqs(solution)                   # residuals and residues sum
    residuals_sum = np.sum(np.abs(residuals))

    if residuals_sum < 1e-14:
        print('\n initial_guess', initial_guess)
        print([i, residuals_sum])

"""best (seed, residue)  : [7028, 9.61991364816208e-16], [9370, 6.41513799031996e-16], [229, 2.8817733270361933e-15], [822, 5.0057674612541584e-15]"""

##### Creating POVM and checking ortho. rel

In [13]:
sol7028 = seed_sol(7028)
(sol7028.solution, sol7028.res)

(array([-9.23992791e-02, -8.97959585e-02,  8.97959585e-02,  8.97959585e-02,
        -8.97959585e-02,  8.97959585e-02, -8.97959585e-02, -3.84555521e-01,
        -3.73720790e-01,  3.73720790e-01,  3.73720790e-01, -3.73720790e-01,
         3.73720790e-01, -3.73720790e-01,  1.21361268e+00,  1.17941952e+00,
        -1.17941952e+00, -1.17941952e+00,  1.17941952e+00, -1.17941952e+00,
         1.17941952e+00, -1.40382467e+00, -1.36427235e+00,  1.36427235e+00,
         1.36427235e+00, -1.36427235e+00,  1.36427235e+00, -1.36427235e+00,
         1.71248577e-27, -6.17394907e+00,  1.21739491e+01,  6.17394907e+00,
        -1.73949065e-01,  6.17394907e+00, -1.73949065e-01]),
 9.61991364816208e-16)

In [17]:
povm7028 = sol7028.full_POVM()
for i in range(9):
    print(np.round(povm7028[i], 3).tolist())

[0j, (0.408+0j), (-0.408+0j), (0.816+0j), 0j, 0j, 0j, 0j, 0j]
[(-0.408+0j), 0j, (0.408+0j), (0.204+0j), (0.791+0j), 0j, 0j, 0j, 0j]
[(0.408+0j), (-0.408+0j), 0j, (0.204+0j), (0.158+0j), (-0.038+0j), (-0.157+0j), (0.495+0j), (-0.573+0j)]
[0j, (-0.204+0.354j), (0.204+0.354j), (0.204+0j), (-0.158-0.183j), (-0.037+0j), (-0.153+0j), (0.481+0j), (-0.557+0j)]
[(-0.408+0j), 0j, (-0.204-0.354j), (-0.102-0.177j), (-0.079+0.228j), (0.037+0j), (0.153+0j), (-0.481+0j), (0.557+0j)]
[(0.408+0j), (0.204-0.354j), 0j, (-0.102+0.177j), (0.237-0.046j), (0.037+0j), (0.153+0j), (-0.481+0j), (0.557+0j)]
[0j, (-0.204-0.354j), (0.204-0.354j), (0.204+0j), (-0.158+0.183j), (-0.037+0j), (-0.153+0j), (0.481+0j), (-0.557+0j)]
[(-0.408+0j), 0j, (-0.204+0.354j), (-0.102+0.177j), (-0.079-0.228j), (0.037+0j), (0.153+0j), (-0.481+0j), (0.557+0j)]
[(0.408+0j), (0.204+0.354j), 0j, (-0.102-0.177j), (0.237+0.046j), (-0.037+0j), (-0.153+0j), (0.481+0j), (-0.557+0j)]


In [20]:
rel7028 = POVM_relations(povm7028)
rel7028.all()

{'11': 1.0000000000000002,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 0,
 '16': 0,
 '17': 0,
 '18': 0,
 '19': 0,
 '22': 1.0000000000000004,
 '23': 0,
 '24': 0,
 '25': 0,
 '26': 0,
 '27': 0,
 '28': 0,
 '29': 0,
 '33': 1.0000000000000002,
 '34': 0.7047120247994255,
 '35': 0.7830951894845303,
 '36': 0.5132065491623172,
 '37': 0.7047120247994257,
 '38': 0.7830951894845303,
 '39': 0.7047120247994255,
 '44': 1.0000000000000002,
 '45': 0.7881060278357926,
 '46': 0.7881060278357926,
 '47': 0.4371625682868003,
 '48': 0.4932882862316249,
 '49': 0.6009252125773318,
 '55': 1.0000000000000002,
 '56': 0.36055512754639896,
 '57': 0.49328828623162496,
 '58': 0.6009252125773317,
 '59': 0.7023769168568496,
 '66': 1.0,
 '67': 0.5686240703077328,
 '68': 0.43716256828679995,
 '69': 0.4932882862316248,
 '77': 1.0000000000000002,
 '78': 0.7881060278357928,
 '79': 0.3605551275463992,
 '88': 1.0000000000000002,
 '89': 0.7881060278357928,
 '99': 1.0000000000000002}

## Two more possible approaches: directly minimize lagrange function without using equations. 

In [48]:
class povm_create:                  # creates POVM object given a list of 28 coefficients (the unknown variables)
    def __init__(self, coeffs):     # coeffs is a list of 7.4 = 28 elements
        self.coeffs = coeffs
    @property
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        sol = self.coeffs
        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7
        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy 
        for i in range(2):                              # first two vecs are known
            POVM_normalized_9d[i].extend([0] *4)
        for part in solution_add:                       # adding (already normalized) elements from the solution to make the remaining vectors 9d
            for i in range(7):
                # POVM_normalized_9d[i+2].append(part[i])
                POVM_normalized_9d[i+2].append((1/np.sqrt(6))*(part[i]))        # normalizing coming from the fixing of the lagrangian eqns
        return POVM_normalized_9d



class POVM_gram_matrix:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
    def all(self, clean=True, threshold=1e-12, absolutes=True):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        # Initialize a 9x9 matrix of zeros
        gram_matrix = np.zeros((9, 9))     # initialize 9x9 empty matrix
        for i in range(9):
            for j in range(i, 9):
                value = np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])
                # Fill both [i, j] and [j, i] to maintain symmetry
                gram_matrix[i, j] = value
                gram_matrix[j, i] = np.conjugate(value)
        
        # Apply absolutes and threshold cleaning if needed
        if absolutes:
            gram_matrix = np.abs(gram_matrix)
        if clean:
            gram_matrix[gram_matrix < threshold] = 0
        return gram_matrix
    


In [49]:
def Relation_Res(vars):     # vars should be a list of 28 elements
    if len(vars) != 28:
        raise ValueError("Expected a list of 28 elements for vars.")
    C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99 = vars
    povm = povm_create(vars).full_POVM             # create POVM object and use that to get the POVM list
    povm_gram_matrix = POVM_gram_matrix(povm).all()    # get the gram matrix of the POVM
    residue_ortho = (1/2 )* np.sum(np.abs(povm_gram_matrix - np.eye(9)))    # calculate the residue
    return residue_ortho

In [53]:
# minimize this function to get the best POVM
# if we get res = .28, avg overlap is .01; which means 89.43 degrees between the vectors on average.
for i in range(200, 600):
    np.random.seed(i)
    ig = np.random.rand(28)
    sol = minimize(Relation_Res, ig, method='CG', tol=1e-12)
    if sol.fun < .7:
        print( i, sol.fun)
        print( sol.x)

206 0.6000000378351897
[ 0.48080811 -0.53257728 -0.59567413  0.34927248  0.7034957   1.74418237
  0.31842766 -0.11179924  0.82035705  1.04913508  1.78180546 -0.26641333
  0.16802538 -0.45284577 -0.69852437  0.25911248  0.66969563  0.04209237
  0.59417256  0.41573044  1.74185805  1.69363165 -0.04205216  1.22310902
 -0.31844351 -0.32650975  0.39593374  0.2438546 ]
211 0.6727781253016387
[ 0.65365957 -0.54351622 -0.04555958 -0.74794763  0.72367838  1.33398874
  1.3827172   0.51314291  0.7397426   1.83402216  0.65156316 -0.32840557
 -0.09294017  0.57637592 -1.66066694  0.58320242 -0.14761475  0.6040218
  0.59494173  0.07095377  0.97998414  0.38935697  0.0413506   0.11178959
  1.43220022  0.20660019  1.26759685 -0.44217076]
225 0.600000055234416
[-0.62160641 -0.03008858 -0.43666901  1.04440953  0.7696762   1.35322843
  0.49375631  1.00196931 -0.43766884  0.48787315 -0.79500578  0.4538715
  1.05558751  1.33649707 -0.41115967  0.90287598  1.55009865  0.96060784
 -0.15909639 -0.3073892   0.693

In [55]:
np.arccos(.60/21)*(180/np.pi)


88.36275492222224

In [80]:
ss =[ 0.75943842 , 0.63041357,  1.34526272 , 1.47362299 ,-0.36159751 , 0.46593514,-0.42186342,  0.12758155, \
     -0.48045654, -0.58975767,  0.31624731,  0.86137059, 1.73341509,  0.69982853, -1.21069275,  0.5744697,  \
      0.68899933,  0.48326614, 0.38980569, -0.09435017,  1.32207525,  1.24145047, -0.25933907,  0.87620439, \
     -0.94597927,  0.02952245,  0.41143011,  0.99216282,]

p = povm_create(ss).full_POVM

g = POVM_gram_matrix(p).all()

print(np.round(g,6)[3][3])
print(np.round(g,6)[6][6])

np.round(g,6)

0.604255
0.604255


array([[1.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      ],
       [0.      , 1.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      ],
       [0.      , 0.      , 1.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.604255, 0.      , 0.      ,
        0.204255, 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 1.      , 0.      ,
        0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 1.      ,
        0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.204255, 0.      , 0.      ,
        0.604255, 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 1.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 1.      ]])